In [1]:
import random
import numpy as np

from torch import nn

# custom libs
from libs.PretrainedModels import AlexNet_cc, SqueezeNet_cc, ResNet18_cc, SqueezeNet1_cc
from libs.utils import get_model_name, init_model, load_model
from libs.Dataset import dst

In [2]:
random.seed(1996)
np.random.seed(1996)

## Dataset and dataLoaders

In [3]:
# parameters for dataloaders
batch_size=32
num_workers=2
drop_last=True

Create data loader

In [4]:
dst.create_data_loader(batch_size=batch_size, num_workers=num_workers, drop_last=drop_last)

## Training

In [5]:
learning_rates = {
    'alexnet': 0.00227, 
    'squeezenet': 0.002,
    'squeezenet1_1': 0.00282, 
    'resnet18': 0.00255
}

Start to make a training of 5 epochs to see the differences between models

In [6]:
num_epochs = 50
momentum = 0.99
criterion = nn.CrossEntropyLoss()

train_from_epoch = 0
save_each = 10
resume_global_step_from = 0

### AlexNet

In [7]:
alexnet = init_model(creator=AlexNet_cc(), model_name=get_model_name(model_name='AlexNet', lr=str(learning_rates['alexnet'])), feature_extract=True, use_pretrained=True)
alexnet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['alexnet'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from)

Initializing: AlexNet__lr=0.00227
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 2.5124 Acc: 0.8704
Epoch 1/49
----------
validation Loss: 1.2292 Acc: 0.9284
Epoch 2/49
----------
validation Loss: 1.3616 Acc: 0.9310
Epoch 3/49
----------
validation Loss: 0.8292 Acc: 0.9497
Epoch 4/49
----------


### SqueezeNet 1.0

In [7]:
# to do 
squeezenet = init_model(creator=SqueezeNet_cc(), model_name=get_model_name(model_name='SqueezeNet', lr=str(learning_rates['squeezenet'])), feature_extract=True, use_pretrained=True)
squeezenet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from )

Initializing: SqueezeNet__lr=0.002
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 0.3049 Acc: 0.8925
Epoch 1/49
----------
validation Loss: 0.2364 Acc: 0.9146
Epoch 2/49
----------
validation Loss: 0.2167 Acc: 0.9230
Epoch 3/49
----------
validation Loss: 0.2004 Acc: 0.9242
Epoch 4/49
----------
validation Loss: 0.1895 Acc: 0.9333
Epoch 5/49
----------
validation Loss: 0.1715 Acc: 0.9356
Epoch 6/49
----------
validation Loss: 0.1639 Acc: 0.9432
Epoch 7/49
----------
validation Loss: 0.1671 Acc: 0.9432
Epoch 8/49
----------
validation Loss: 0.1722 Acc: 0.9333
Epoch 9/49
----------
validation Loss: 0.1421 Acc: 0.9531
Epoch 10/49
----------
validation Loss: 0.1641 Acc: 0.9455
Epoch 11/49
----------
validation Loss: 0.1306 Acc: 0.9569
Epoch 12/49
----------
validation Loss: 0

### SqueezeNet 1.1

In [ ]:
squeezenet1_1 = init_model(creator=SqueezeNet1_cc(), model_name=get_model_name(model_name='SqueezeNet1_1', lr=str(learning_rates['squeezenet1_1'])), feature_extract=True, use_pretrained=True)
squeezenet1_1.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet1_1'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from )

### ResNet-18

In [8]:
resnet18 = init_model(creator=ResNet18_cc(), model_name=get_model_name(model_name='ResNet18', lr=str(learning_rates['resnet18'])), feature_extract=True, use_pretrained=True)
resnet18.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['resnet18'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from )

Initializing: ResNet18__lr=0.00255
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/49
----------
validation Loss: 0.2604 Acc: 0.9272
Epoch 1/49
----------
validation Loss: 0.3098 Acc: 0.9333
Epoch 2/49
----------
validation Loss: 0.2485 Acc: 0.9402
Epoch 3/49
----------
validation Loss: 0.2296 Acc: 0.9501
Epoch 4/49
----------
validation Loss: 1.0046 Acc: 0.8537
Epoch 5/49
----------
validation Loss: 0.3299 Acc: 0.9375
Epoch 6/49
----------
validation Loss: 0.7834 Acc: 0.8880
Epoch 7/49
----------
validation Loss: 0.2959 Acc: 0.9417
Epoch 8/49
----------
validation Loss: 0.3085 Acc: 0.9451
Epoch 9/49
----------
validation Loss: 0.4263 Acc: 0.9211
Epoch 10/49
----------
validation Loss: 0.1576 Acc: 0.9550
Epoch 11/49
----------
validation Loss: 0.3311 Acc: 0.9314
Epoch 12/49
----------
validation Loss: 0

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Test accuracy -- todo

In [ ]:
### il migliore dopo 50 epoche lo provi con batch size diverso, fai in un nuovo notebook i calcoli con batch a 64 e 128 e vedi se ci sono miglioramenti
### poi continua quello salvato con altre ulteriori 100 epoche

In [7]:
trained_models = {
    'alexnet': 'models/AlexNet__lr=0.00227-50.pth', 
    'squeezenet': 'models/SqueezeNet__lr=0.002-50.pth',
    'squeezenet1_1': 'models/SqueezeNet1_1__lr=0.00282/SqueezeNet1_1__lr=0.00282-50.pth', 
    'resnet18': 'models/ResNet18__lr=0.00255-50.pth'
}

concreteCreators = {
    'alexnet': AlexNet_cc(),
    'squeezenet': SqueezeNet_cc(),
    'squeezenet1_1': SqueezeNet1_cc(),
    'resnet18': ResNet18_cc()
}

In [9]:
for name in trained_models:
    model = load_model(creator=concreteCreators[name], model_name=get_model_name(model_name=name, lr=str(learning_rates[name])), num_classes=3, model_path=trained_models[name])
    model.test_accuracy(dataset=dst)

Initializing: alexnet__lr=0.00227
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of alexnet__lr=0.00227: 83.97%
Initializing: squeezenet__lr=0.002
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of squeezenet__lr=0.002: 86.53%
Initializing: squeezenet1_1__lr=0.00282
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of squeezenet1_1__lr=0.00282: 86.20%
Initializing: resnet18__lr=0.00255
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of resnet18__lr=0.00255: 79.12%


### Continue training squeezenet (best model)

In [10]:
num_epochs = 20
momentum = 0.99
criterion = nn.CrossEntropyLoss()

train_from_epoch = 50
save_each = 5
resume_global_step_from = 458176 # need for tensorboard

In [11]:
best_model = load_model(creator=concreteCreators['squeezenet'], model_name=get_model_name(model_name='SqueezeNet', lr=str(learning_rates['squeezenet'])), num_classes=3, model_path='models/SqueezeNet__lr=0.002-50.pth' )
best_model.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from)

Initializing: SqueezeNet__lr=0.002
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Feature extracting
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Epoch 0/19
----------
validation Loss: 0.1028 Acc: 0.9619
Epoch 1/19
----------
validation Loss: 0.0961 Acc: 0.9642
Epoch 2/19
----------
validation Loss: 0.0814 Acc: 0.9703
Epoch 3/19
----------
validation Loss: 0.0878 Acc: 0.9703
Epoch 4/19
----------
validation Loss: 0.1102 Acc: 0.9634
Epoch 5/19
----------
validation Loss: 0.0816 Acc: 0.9684
Epoch 6/19
----------
validation Loss: 0.0871 Acc: 0.9684
Epoch 7/19
----------
validation Loss: 0.0807 Acc: 0.9726
Epoch 8/19
----------
validation Loss: 0.0864 Acc: 0.9718
Epoch 9/19
----------
validation Loss: 0.1055 Acc: 0.9665
Epoch 10/19
----------
validation Loss: 0.0924 Acc: 0.9722


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [13]:
model = load_model(creator=concreteCreators['squeezenet'], model_name=get_model_name(model_name='SqueezeNet', lr=str(learning_rates['squeezenet'])), num_classes=3, model_path='models/SqueezeNet__lr=0.002-70.pth')
model.test_accuracy(dataset=dst)

Initializing: SqueezeNet__lr=0.002
Loading model using load_state_dict..
Model loaded sucessfully!
Loaded model from file successfully
Accuracy of SqueezeNet__lr=0.002: 87.12%
